In [4]:
import pandas as pd
import numpy as np

# Usage

In [2]:
usage = pd.read_csv('./20&21.csv', encoding='cp949')
usage.head()

,Unnamed: 0,자전거번호,반납일시,이용시간,이용거리
0,0,SPB-04061,2020-01-01,2.0,0.0
1,1,SPB-06686,2020-01-01,1.0,350.0
2,2,SPB-15937,2020-01-01,4.0,800.0
3,3,SPB-14805,2020-01-01,2.0,0.0
4,4,SPB-09038,2020-01-01,4.0,660.0


In [4]:
#필요없는 열 제거
usage.drop(['Unnamed: 0'], axis=1, inplace=True)

#홈페이지 기준 이용시간>240분인 자전거는 도난으로 취급
#이용시간>240인 데이터 삭제 (32331개 데이터 삭제 완료, 전체의 0.05%)
overtime = usage[usage['이용시간'] > 240].index
usage.drop(overtime, inplace=True)

#이용거리=NaN인 데이터는 데이터 자체가 밀려 수집된 것으로 확인
#이용거리=NaN인 값 삭제 (355개 데이터 삭제 완료)
usage = usage.dropna()

#이용시간, 이용거리 정수화
usage = usage.astype({'이용시간' : int, '이용거리' : int}, errors='raise')

#반납일시 datetime type
usage['반납일시'] = pd.to_datetime(usage['반납일시'])

#반납일시 열 이름을 등록일시로
usage.rename(columns={'반납일시': '등록일시'}, inplace=True)

In [27]:
usage.head()

,자전거번호,등록일시,이용시간,이용거리
0,SPB-04061,2020-01-01,2,0
1,SPB-06686,2020-01-01,1,350
2,SPB-15937,2020-01-01,4,800
3,SPB-14805,2020-01-01,2,0
4,SPB-09038,2020-01-01,4,660


In [28]:
usage.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54044163 entries, 0 to 54076493
Data columns (total 4 columns):
 #   Column  Dtype         
---  ------  -----         
 0   자전거번호   object        
 1   등록일시    datetime64[ns]
 2   이용시간    int32         
 3   이용거리    int32         
dtypes: datetime64[ns](1), int32(2), object(1)
memory usage: 1.6+ GB


In [29]:
usage.isna().sum()

자전거번호    0
등록일시     0
이용시간     0
이용거리     0
dtype: int64

# Breakdown

In [5]:
breakdown = pd.read_csv("C:/jupiter_workspace/ml_tp/bike_breakdown.csv", encoding='cp949')
breakdown.head()

,자전거번호,등록일시,고장구분
0,SPB-00108,2015-12-04,체인
1,SPB-00210,2015-12-07,체인
2,SPB-00035,2015-12-09,기타
3,SPB-01024,2015-12-13,기타
4,SPB-01015,2015-12-28,기타


In [6]:
#같은자전거에 같은 날에 여러건 등록된 데이터에 대해서는 첫행만 남기고 제거함
breakdown.drop_duplicates(['자전거번호','등록일시'], keep='first', inplace=True)

#등록일시 datetime type
breakdown['등록일시'] = pd.to_datetime(breakdown['등록일시'])

#고장구분 열 제거
breakdown.drop(['고장구분'], axis=1, inplace=True)


In [32]:
breakdown.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297349 entries, 0 to 337871
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   자전거번호   297349 non-null  object        
 1   등록일시    297349 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 6.8+ MB


In [7]:
breakdown.isna().sum()

자전거번호    0
등록일시     0
dtype: int64

# 이제부터 시작

In [8]:
#쫄리니까 복사본 만들어야지
tuse = usage
tbreak = breakdown

## 고장구분

In [9]:
#냅다 breakdown에 yes 만들기
tbreak['고장구분'] = 'Y'

#usage에 outer join으로 합치기
tmerge = tuse.merge(right=tbreak, how='outer', on=['자전거번호', '등록일시'], suffixes=('_',''))

#null값 있으면 다시 확인해보쟝
tmerge.isna().sum()


자전거번호           0
등록일시            0
이용시간       120169
이용거리       120169
고장구분     53080571
dtype: int64

이용시간 & 이용거리의 결측치 (120169 개) <br>
: 2019년 이전데이터에 대해 usage 데이터가 2020&2021년에 대해서만 수록되어 있어서 발생 -> 제거 <br>

고장구분의 결측치 (53080571 개) <br>
: 고장 안 난 날의 이용기록 -> 보존

In [10]:
#이용시간 & 이용거리의 결측치에 대해서는 삭제
tmerge.dropna(subset=['이용시간', '이용거리'], inplace=True)

#자전거번호, 등록일시, 고장구분이 같은 것 중 마지막 값만 남기고 제거
tmerge.drop_duplicates(['자전거번호', '등록일시', '고장구분'], keep='last', inplace=True)

#모든 행이 동일한 데이터가 있는지 확인
aaa = tmerge.duplicated(subset=['자전거번호', '등록일시', '이용시간', '이용거리', '고장구분'])
aa = aaa.sum()
print(aa) #0 >> 없음

#얘를 다시 usage와 left join
final_df = tuse.merge(right=tmerge, how='left', on=['자전거번호', '등록일시', '이용시간', '이용거리'], suffixes=('_',''))

final_df.head(30)

0


,자전거번호,등록일시,이용시간,이용거리,고장구분
0,SPB-04061,2020-01-01,2,0,NaN
1,SPB-06686,2020-01-01,1,350,NaN
2,SPB-15937,2020-01-01,4,800,NaN
3,SPB-14805,2020-01-01,2,0,NaN
4,SPB-09038,2020-01-01,4,660,NaN
5,SPB-18014,2020-01-01,4,970,NaN
6,SPB-12978,2020-01-01,4,910,NaN
7,SPB-17023,2020-01-01,2,450,NaN
8,SPB-14251,2020-01-01,3,910,NaN
9,SPB-18792,2020-01-01,5,1020,NaN


In [11]:
#고장구분 NaN(고장안남)=0, Y(고장남)=1
final_df['고장구분'] = final_df['고장구분'].fillna(0)
final_df['고장구분'] = final_df['고장구분'].replace('Y', 1)

final_df.head(30)

,자전거번호,등록일시,이용시간,이용거리,고장구분
0,SPB-04061,2020-01-01,2,0,0
1,SPB-06686,2020-01-01,1,350,0
2,SPB-15937,2020-01-01,4,800,0
3,SPB-14805,2020-01-01,2,0,0
4,SPB-09038,2020-01-01,4,660,0
5,SPB-18014,2020-01-01,4,970,0
6,SPB-12978,2020-01-01,4,910,0
7,SPB-17023,2020-01-01,2,450,0
8,SPB-14251,2020-01-01,3,910,0
9,SPB-18792,2020-01-01,5,1020,0


In [13]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54044163 entries, 0 to 54044162
Data columns (total 5 columns):
 #   Column  Dtype         
---  ------  -----         
 0   자전거번호   object        
 1   등록일시    datetime64[ns]
 2   이용시간    int32         
 3   이용거리    int32         
 4   고장구분    int64         
dtypes: datetime64[ns](1), int32(2), int64(1), object(1)
memory usage: 2.0+ GB


## 이용거리=0 인 값 처리

In [ ]:
#이용거리=0이면 60분 미만인 값에 대해서는 같은 분의 평균, 
#60분 이상인 값은 평균 이용거리=3036.95*ln(이용시간)+-4210.63

#데이터가 너무 커서 월별로 자르기
month_list = ['month1', 'month2', 'month3', 'month4', 'month5', 'month6',
                'month7', 'month8', 'month9', 'month10', 'month11', 'month12']

for i in range(12) :
    mon = month_list[i]
    mon = final_df[final_df['등록일시'].dt.month == i+1]
    month_list[i] = mon.reset_index()
    month_list[i].drop(['index'], axis=1, inplace=True)

#방법1

#1~240분의 분 별 이용거리 평균값 리스트 생성
mean_by_time = final_df.groupby(by='이용시간').mean()
mean_by_time = mean_by_time.astype({'이용거리' : int}, errors='raise')
#240행까지 나옴. 1~240분 중 결측치 없음
L_mean = mean_by_time['이용거리'].to_list()

for i in range(final_df.shape[0]) :
    if final_df.iloc[i, 3] == 0 :
        if final_df.iloc[i, 2] <= 60 : #60분 이하면
            usetime = final_df.iloc[i, 2]
            final_df.iloc[i, 3] = L_mean[usetime-1] #같은 이동시간에서의 이동거리 평균값
        else : #60분보다 크면
            usetime = final_df.iloc[i, 2]
            final_df.iloc[i, 3] = 3036.95*np.log(usetime)-4210.63 #회귀식

#1~3월까지 보정한 데이터를 원본데이터에 집어넣어주자
#인덱스번호가 보존되어 있으니 그걸로 replace
index_list = month3.index
final_df.loc[index_list] = month3.loc[index_list]


In [14]:
#방법2

#1~240분의 분 별 이동거리 평균값 데이터프레임 생성
mean_by_time = final_df.groupby(by='이용시간').mean()
mean_by_time = mean_by_time.astype({'이용거리' : int}, errors='raise')
mean_by_time.drop(['고장구분'], axis=1, inplace=True)
#240행까지 나옴. 1~240분 중 결측치 없음

#60~240분 별 이동거리 회귀식 계산결과 데이터프레임 생성
rr=[]
tt=[]
for i in range(60, 241) :
    tt.append(i)
    rr.append(3036.95*np.log(i)-4210.63)

regr_by_time = pd.DataFrame({
    '이용시간' : tt,
    '이용거리' : rr
})

regr_by_time['이용거리'] = regr_by_time['이용거리'].astype(int) #정수화

#원본데이터에서 이용거리=0인 데이터 날리기
zerodist = final_df[final_df['이용거리'] == 0].index
final_df.drop(zerodist, inplace=True)

In [15]:
#이용거리=0인 데이터 보정하고 다시 넣어주기
for mon in month_list :

    con1 = mon['이용거리'] == 0
    con2 = mon['이용시간'] <= 60
    con3 = mon['이용시간'] > 60

    mini = mon[con1 & con2] #이용거리=0, 이용시간<=60분인 데이터
    big = mon[con1 & con3] #이용거리=0, 이용시간>60분이 데이터

    #0만적힌 이동거리 열 제거
    mini.drop(['이용거리'], axis=1, inplace=True)
    big.drop(['이용거리'], axis=1, inplace=True)

    #merge
    mini_merge = mini.merge(right=mean_by_time, how='left', on='이용시간', suffixes=('_',''))
    big_merge = big.merge(right=regr_by_time, how='left', on='이용시간', suffixes=('_',''))

    #컬럼 위치 바꾸기
    mini = mini_merge[['자전거번호', '등록일시', '이용시간', '이용거리', '고장구분']]
    big = big_merge[['자전거번호', '등록일시', '이용시간', '이용거리', '고장구분']]

    #이용거리=0이었던 걸 보정한 데이터프레임 합쳐주기
    final_df=pd.concat([final_df, mini, big], ignore_index=True)

C:\Users\User\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [16]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54044163 entries, 0 to 54044162
Data columns (total 5 columns):
 #   Column  Dtype         
---  ------  -----         
 0   자전거번호   object        
 1   등록일시    datetime64[ns]
 2   이용시간    int32         
 3   이용거리    int32         
 4   고장구분    int64         
dtypes: datetime64[ns](1), int32(2), int64(1), object(1)
memory usage: 1.6+ GB


### 나중에 회귀선 그래프 사진 넣기~!!

In [22]:
#이용거리 열 int 타입으로
final_df = final_df.astype({'이용거리' : int}, errors='raise')

#자전거이름, 대여일시 순으로 정렬
final_df = final_df.sort_values(by=['자전거번호', '등록일시'], ascending=True)
final_df = final_df.reset_index()
final_df.drop(['index'], axis=1, inplace=True)

final_df.head()

,자전거번호,등록일시,이용시간,이용거리,고장구분
0,SPB-00003,2020-04-01,3,360,0
1,SPB-00003,2020-04-01,22,1961,0
2,SPB-00003,2020-04-01,38,3273,0
3,SPB-00003,2020-04-01,19,1696,0
4,SPB-00003,2020-04-01,6,609,0


In [23]:
#쫄리니까 csv파일 하나 만들어주기~
final_df.to_csv("final1.csv", mode='w',encoding='euc-kr')

In [5]:
final_df = pd.read_csv("./final1.csv", encoding='cp949')

final_df['등록일시'] = pd.to_datetime(final_df['등록일시'])
final_df.drop(['Unnamed: 0'], axis=1, inplace=True)
final_df.head()

,Unnamed: 0,자전거번호,등록일시,이용시간,이용거리,고장구분
0,0,SPB-00003,2020-04-01,3,360,0
1,1,SPB-00003,2020-04-01,22,1961,0
2,2,SPB-00003,2020-04-01,38,3273,0
3,3,SPB-00003,2020-04-01,19,1696,0
4,4,SPB-00003,2020-04-01,6,609,0


## 누적이용시간, 누적이용거리, 누적이용횟수

In [6]:
#누적 이용시간 & 누적 이동거리, 이용횟수 빈 열 생성
final_df.loc[:, '누적이용시간'] = np.nan
final_df.loc[:, '누적이용거리'] = np.nan
final_df.loc[:, '누적이용횟수'] = np.nan

#이용시간과 이동거리를 계속 합하다가 고장구분에 1을 만나면 기록
#자전거 넘버가 바뀌면 0으로 리셋, 다시시작

final_df['누적이용시간'] = final_df.groupby(by=['자전거번호'])['이용시간'].apply(lambda x: x.cumsum())
final_df['누적이용거리'] = final_df.groupby(by=['자전거번호'])['이용거리'].apply(lambda x: x.cumsum())
final_df['누적이용횟수'] = final_df.groupby(by=['자전거번호']).cumcount()+1


In [7]:
final_df.head()

,Unnamed: 0,자전거번호,등록일시,이용시간,이용거리,고장구분,누적이용시간,누적이용거리,누적이용횟수
0,0,SPB-00003,2020-04-01,3,360,0,3,360,1
1,1,SPB-00003,2020-04-01,22,1961,0,25,2321,2
2,2,SPB-00003,2020-04-01,38,3273,0,63,5594,3
3,3,SPB-00003,2020-04-01,19,1696,0,82,7290,4
4,4,SPB-00003,2020-04-01,6,609,0,88,7899,5


In [8]:
#이용강도 = 단위시간 당 평균적으로 얼마를 달렸는지
#평균이용거리 = 1회 당 평균적으로 얼마를 달렸는지
final_df['이용강도'] = final_df['누적이용거리'] / final_df['누적이용시간']
final_df['평균이용거리'] = final_df['누적이용거리'] / final_df['누적이용횟수']

#모두 정수화
final_df = final_df.astype({'이용강도' : int, '평균이용거리' : int}, errors='raise')



In [11]:
final_df.head()

,자전거번호,등록일시,이용시간,이용거리,고장구분,누적이용시간,누적이용거리,누적이용횟수,이용강도,평균이용거리
0,SPB-00003,2020-04-01,3,360,0,3,360,1,120,360
1,SPB-00003,2020-04-01,22,1961,0,25,2321,2,92,1160
2,SPB-00003,2020-04-01,38,3273,0,63,5594,3,88,1864
3,SPB-00003,2020-04-01,19,1696,0,82,7290,4,88,1822
4,SPB-00003,2020-04-01,6,609,0,88,7899,5,89,1579


## 자전거 수명

In [23]:
#자전거 수명 구하기

#자전거 번호 별 첫 대여일만 남긴 프레임
first_df = final_df.drop_duplicates(['자전거번호'], keep='first') #첫 대여일만 남기기
first_df = first_df.iloc[:, :2] #자전거번호랑 등록일시만 남기기
first_df.rename(columns={'등록일시': '첫 대여일'}, inplace=True) #열 이름 바꾸기

#두 데이터 합치기
final_df = final_df.merge(right=first_df, how='left', on='자전거번호', suffixes=('_',''))

#수명 구하기
final_df['나이'] = final_df['등록일시'] - final_df['첫 대여일']

final_df.tail()
#결측치 없음

,자전거번호,등록일시,이용시간,이용거리,고장구분,누적이용시간,누적이용거리,누적이용횟수,이용강도,평균이용거리,첫 대여일,나이
54044158,SPB-90005,2020-08-19,32,2846,0,36,3340,3,92,1113,2020-06-25,55 days
54044159,SPB-90005,2020-08-19,21,1857,0,57,5197,4,91,1299,2020-06-25,55 days
54044160,SPB-90005,2020-08-19,5,526,0,62,5723,5,92,1144,2020-06-25,55 days
54044161,SPB-90005,2020-08-20,2,247,0,64,5970,6,93,995,2020-06-25,56 days
54044162,SPB-90005,2020-08-20,45,3878,0,109,9848,7,90,1406,2020-06-25,56 days


## 여름횟수

In [52]:
summer_df = final_df.loc[:, ['등록일시', '첫 대여일']]
summer_df

,등록일시,첫 대여일
0,2020-04-01,2020-04-01
1,2020-04-01,2020-04-01
2,2020-04-01,2020-04-01
3,2020-04-01,2020-04-01
4,2020-04-01,2020-04-01
...,...,...
54044158,2020-08-19,2020-06-25
54044159,2020-08-19,2020-06-25
54044160,2020-08-19,2020-06-25
54044161,2020-08-20,2020-06-25


In [69]:
#여름 횟수 구하는 함수
def get_summer(i) :
    #첫대여일~등록일시 까지의 월
    k = pd.period_range(start=summer_df.iloc[i, 1], end=summer_df.iloc[i, 0], freq='M')

    #프레임 만들어주기
    kk = pd.DataFrame(index=k, data=[0]*len(k))

    #월이 7월인 것만 빼오기
    kkk = list(filter(lambda x: x==7, kk.index.month))

    #7월의 개수 세기
    return(len(kkk))

In [80]:
test = final_df.tail(30) #테스트 데이터프레임

index_list2 = test.index.to_series() #인덱스만 담은 시리즈

#df에 '여름'이라는 열을 만들고 여름 개수를 넣자
test['여름'] = index_list2.map(lambda x: get_summer(x)) 

test

C:\Users\User\AppData\Local\Temp/ipykernel_9540/1171854937.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['여름'] = index_list2.map(lambda x: get_summer(x))


,자전거번호,등록일시,이용시간,이용거리,고장구분,누적이용시간,누적이용거리,누적이용횟수,이용강도,평균이용거리,첫 대여일,나이,여름
54044133,SPB-84251,2021-12-14,21,2760,0,569,40220,23,70,1748,2021-12-02,12 days,0
54044134,SPB-84251,2021-12-15,19,4420,0,588,44640,24,75,1860,2021-12-02,13 days,0
54044135,SPB-84251,2021-12-23,5,1050,0,593,45690,25,77,1827,2021-12-02,21 days,0
54044136,SPB-84251,2021-12-25,5,1730,0,598,47420,26,79,1823,2021-12-02,23 days,0
54044137,SPB-84252,2021-12-11,52,4560,0,52,4560,1,87,4560,2021-12-11,0 days,0
54044138,SPB-84252,2021-12-11,18,2540,0,70,7100,2,101,3550,2021-12-11,0 days,0
54044139,SPB-84252,2021-12-11,84,13760,0,154,20860,3,135,6953,2021-12-11,0 days,0
54044140,SPB-84252,2021-12-12,48,4230,0,202,25090,4,124,6272,2021-12-11,1 days,0
54044141,SPB-84252,2021-12-12,25,4180,0,227,29270,5,128,5854,2021-12-11,1 days,0
54044142,SPB-84252,2021-12-12,177,15770,0,404,45040,6,111,7506,2021-12-11,1 days,0


# 얘네는 대충 연습한거! 나 경계선 ---------------------------

In [ ]:
bikes = []
bikes = final_df['자전거번호'].unique()

for bike in bikes :
    cum_time = 0
    cum_dist = 0
    cum_use = 0

    bike_index = final_df[final_df['자전거번호'] == bike].index

    for k in bike_index :

        cum_time += final_df.iloc[k, 2] #누적이용시간
        cum_dist += final_df.iloc[k, 3] #누적이용거리
        cum_use += 1 #누적이용횟수

        if final_df.iloc[k, 4] == 1 : #고장났으면 이제까지 누적값 입력
            final_df.iloc[k, 5] == cum_time
            final_df.iloc[k, 6] == cum_dist
            final_df.iloc[k, 7] == cum_use

In [51]:
tmerge[(tmerge['이용시간'] == 0)]

#aaa = tmerge.duplicated(subset=['자전거번호', '등록일시', '이용시간', '이용거리', '고장구분'])
#aa = aaa.sum()
#print(aa)

,자전거번호,등록일시,이용시간,이용거리,고장구분


In [50]:
#test = final_df.tail()

def get_summer(i) :
    #첫대여일~등록일시 까지의 월
    k = pd.period_range(start=final_df.iloc[i, 1], end=final_df.iloc[i, 0], freq='M')

    #프레임 만들어주기
    kk = pd.DataFrame(index=k, data=[0]*len(k))

    #월이 7월인 것만 빼오기
    kkk = list(filter(lambda x: x==7, kk.index.month))

    #7월의 개수 세기
    return(len(kkk))


PeriodIndex(['2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08',
             '2020-09', '2020-10', '2020-11', '2020-12', '2021-01', '2021-02',
             '2021-03', '2021-04', '2021-05', '2021-06', '2021-07', '2021-08'],
            dtype='period[M]')


AttributeError: 'PeriodIndex' object has no attribute 'index'